# §2.3 Symbolic Data

### Exercise 2.53

In [1]:
(list 'a 'b 'c)

(a b c)

In [2]:
(list (list 'george))

((george))

In [3]:
(cdr '((x1 x2) (y1 y2)))

((y1 y2))

In [4]:
(cadr '((x1 x2) (y1 y2)))

(y1 y2)

In [5]:
(pair? (car '(a short list))) ; ~~> (pair? 'a) thus #f

#f

In [6]:
(memq 'red '((red shoes) (lue socks))) ; memq doesn't deal with nested list, thus #f

#f

In [7]:
(memq 'red '(red shoes blue socks))

(red shoes blue socks)

### Exercise 2.54

In [8]:
(define (my-equal? xs ys)
  (cond
    [(and (pair? xs) (pair? ys))
     (and (my-equal? (car xs) (car ys))
          (my-equal? (cdr xs) (cdr ys)))]
    [else (eq? xs ys)]))

In [9]:
(list
 (my-equal? 'a 'a)
 (my-equal? 'a 'b)
 (my-equal? 3 3)
 (my-equal? 4 3))

(#t #f #t #f)

In [10]:
(list
 (my-equal? '(a b c) '(a b c))
 (my-equal? '(a (b) c) '(a (b) c))
 (my-equal? '(a (b) c) '(a (b d) c))
 (my-equal? '(a b c) 'a)
 (my-equal? '(a b c) '(b c a)))

(#t #t #f #f #f)

### Exercise 2.55

In [11]:
(car ''abracadabra) ; ~~> (car '(quote abracadabra)) ~~> 'quote

quote

## §§2.3.2 Example: Symbolic Differentiation

In [12]:
(if #t 0
  (list ; we just wish that we have the following; we just wish...
   (variable? e)
   (same-variable? v1 v2)
   
   (sum? e)
   (addend e)
   (augend e)
   (make-sum a1 a2)
   
   (product? e)
   (multiplier e)
   (multiplicant e)
   (make-product m1 m2)))

0

In [13]:
(define (deriv exp var)
  (cond [(number? exp) 0]
        [(variable? exp)
         (if (same-variable? exp var) 1 0)]
        [(sum? exp)
         (make-sum (deriv (addend exp) var)
                   (deriv (augend exp) var))]
        [(product? exp)
         (make-sum
          (make-product (multiplier exp)
                        (deriv (multiplicand exp) var))
          (make-product (multiplicand exp)
                        (deriv (multiplier exp) var)))]
        [else (error "what are you feeding me? is that dog food?! -- DERIV") exp]))

In [14]:
;; we use symbols to represent variables
(define (variable? e) (symbol? e))
(define (same-variable? v1 v2)
  (and (variable? v1) (variable? v2) (eq? v1 v2)))

;; we use lists of the form ('+ e1 e2) to represent sums
(define (sum? e)
  (and (pair? e) (eq? (car e) '+)))
(define (make-sum a1 a2) (list '+ a1 a2))
(define (addend e) (cadr e))
(define (augend e) (caddr e))

;; we use lists of the form ('* e1 e2) to represent products
(define (product? e)
  (and (pair? e) (eq? (car e) '*)))
(define (make-product m1 m2) (list '* m1 m2))
(define (multiplier e) (cadr e))
(define (multiplicand e) (caddr e))

In [15]:
(deriv '(+ x 3) 'x)

(+ 1 0)

In [16]:
(deriv '(* (* x y) (+ x 3)) 'x)

(+ (* (* x y) (+ 1 0)) (* (+ x 3) (+ (* x 0) (* y 1))))

In [17]:
(define (make-sum a1 a2)
  (cond [(and (number? a1) (number? a2)) (+ a1 a2)]
        [(eqv? 0 a1) a2]
        [(eqv? 0 a2) a1]
        [else (list '+ a1 a2)]))

(define (make-product m1 m2)
  (cond [(and (number? m1) (number? m2)) (* m1 m2)]
        [(eqv? 1 m1) m2]
        [(eqv? 1 m2) m1]
        [(or (eqv? 0 m1) (eqv? 0 m2)) 0]
        [else (list '* m1 m2)]))

In [18]:
(deriv '(+ x 3) 'x)

1

In [19]:
(deriv '(* x y) 'x)

y

In [20]:
(deriv '(* (* x y) (+ x 3)) 'x)

(+ (* x y) (* (+ x 3) y))

In [21]:
(deriv '(* (* x y) (+ x 3)) 'x)

(+ (* x y) (* (+ x 3) y))

### Exercise 2.56

In [22]:
(define (deriv exp var)
  (cond [(number? exp) 0]
        [(variable? exp)
         (if (same-variable? exp var) 1 0)]
        [(sum? exp)
         (make-sum (deriv (addend exp) var)
                   (deriv (augend exp) var))]
        [(product? exp)
         (make-sum
          (make-product (multiplier exp)
                        (deriv (multiplicand exp) var))
          (make-product (multiplicand exp)
                        (deriv (multiplier exp) var)))]
        [(exponentiation? exp)
         (make-product (exponent exp)
                       (make-product (make-exponentiation (base exp) (- (exponent exp) 1))
                                     (deriv (base exp) var)))]
        [else (error "what are you feeding me? is that dog food?! -- DERIV") exp]))

;; we use lists of form '(** x a) to express exponentiations.
;; we restrict a to be a number
(define (exponentiation? e) (and (pair? e) (eq? (car e) '**)))
(define (base e) (cadr e))
(define (exponent e) (caddr e))
(define (make-exponentiation x a)
  (cond [(eqv? x 0) 0]
        [(eqv? x 1) 1]
        [(eqv? a 0) 1]
        [(eqv? a 1) x]
        [else (list '** x a)]))

In [23]:
(deriv '(** x 3) 'x)

(* 3 (** x 2))

In [24]:
(deriv '(** (+ (** x 6) (* 2 x)) 3) 'x)

(* 3 (* (** (+ (** x 6) (* 2 x)) 2) (+ (* 6 (** x 5)) 2)))

### Exercise 2.57

In [25]:
%include "/library/prelude.scm"
;; we are going to use `fn` and `rec` defined there
;; the file also defines some high-order list functions

#### Straightforward implementation

Since we know that

\begin{align}
\frac{\partial}{\partial x} (f_1(x) + f_2(x) + \cdots + f_n(x))
  \; = \; & f_1'(x) + f_2'(x) + \cdots + f_n'(x) \\
\frac{\partial}{\partial x} (f_1(x) \cdot f_2(x) \cdot \cdots \cdot f_n(x))
  \; = \; & f_1'(x) \cdot f_2(x) \cdot \cdots \cdot f_n(x) \\
     + \; & f_2'(x) \cdot f_1(x) \cdot \cdots \cdot f_n(x) \\
     + \; & \cdots \\
     + \; & f_n'(x) \cdot f_1(x) \cdot f_2(x) \cdot \cdots \cdot f_{n-1}(x)
\end{align}

It's quite straightforward to re-write the `deriv` function using higher order functions on lists (e.g. `map`).

The trick is that we want a higher order function in implementing the derivative of products that pick each element (with the else of the list) from a list, which we call it `pick` (and a corresponding map-function, `pick-map`) and will implement it later.

In [26]:
(define (deriv exp var)
  (define (D e) (deriv e var))
  (cond [(number? exp) 0]
        [(variable? exp)
         (if (same-variable? exp var) 1 0)]
        [(sum? exp)
         (make-sum (map D (terms exp)))]
        [(product? exp)
         (make-sum
          (pick-map (fn (e es) (make-product (cons (D e) es)))
                    (terms exp)))]
        [(exponentiation? exp)
         (make-product-lit (exponent exp)
                           (make-product-lit
                             (make-exponentiation (base exp) (- (exponent exp) 1))
                             (deriv (base exp) var)))]
        [else (error "what are you feeding me? is that dog food?! -- DERIV") exp]))

;; we use lists of the form ('+ e1 e2 ...) to represent sums
(define (sum? e)
  (and (pair? e) (eq? (car e) '+)))
(define (make-sum terms) (cons '+ terms))
(define (make-sum-lit a1 a2) (make-sum (list a1 a2)))
(define (terms e) (cdr e))

;; we use lists of the form ('* e1 e2 ...) to represent products
(define (product? e)
  (and (pair? e) (eq? (car e) '*)))
(define (make-product factors) (cons '* factors))
(define (make-product-lit m1 m2) (make-product (list m1 m2)))
(define (factors e) (cdr e))

#### `pick` and `pick-map`

In [27]:
(define (pick xs)
  (define (F front tail acc)
    (if (null? tail) acc
        (F (cons (car tail) front)
           (cdr tail)
           (cons (cons (car tail) (append (reverse front) (cdr tail))) acc))))
  (reverse (F '() xs '())))
(define (pick-map f xs)
  (map (fn (p) (f (car p) (cdr p))) (pick xs)))

In [28]:
(pick (range 5))

((0 1 2 3 4) (1 0 2 3 4) (2 0 1 3 4) (3 0 1 2 4) (4 0 1 2 3))

In [29]:
(pick-map (fn (x xs) (cons xs x)) (range 5))

(((1 2 3 4) . 0) ((0 2 3 4) . 1) ((0 1 3 4) . 2) ((0 1 2 4) . 3) ((0 1 2 3) . 4))

#### Refinements of constructors `make-sum` and `make-product`

In [30]:
(define (make-sum terms)
  (define filtered (filter-out (eqv-pred 0) terms))
  (cond [(null? filtered) 0]
        [(forall number? terms) (sum filtered)]
        [(singleton? filtered) (car filtered)]
        [else (cons '+ filtered)]))
(define (make-product factors)
  (define (prod xs) (foldl * 1 xs))
  (define filtered (filter-out (eqv-pred 1) factors))
  (cond [(null? filtered) 1]
        [(forall number? filtered) (prod filtered)]
        [(exists (eqv-pred 0) filtered) 0]
        [(singleton? filtered) (car filtered)]
        [else (cons '* filtered)]))

(define (filter-out pred xs) (filter (negate pred) xs))
(define (eqv-pred x) (fn (y) (eqv? x y)))

#### Tryouts

In [31]:
(deriv '(** (+ (** x 6) (* 2 x)) 3) 'x)

(* 3 (* (** (+ (** x 6) (* 2 x)) 2) (+ (* 6 (** x 5)) 2)))

In [32]:
(deriv '(+ (* 3 x) (* 4 x) (* (** x 2) y)) 'x)

(+ 3 4 (* (* 2 x) y))

In [33]:
(deriv '(* (+ (* 3 x) (* 4 x) (* (** x 2) y)) y x) 'x)

(+ (* (+ 3 4 (* (* 2 x) y)) y x) (* (+ (* 3 x) (* 4 x) (* (** x 2) y)) y))

### Exercise 2.58 (a)

The key for this exercise is the `interpose` higher-order list function. What it does is to insert a /separator/ in between of every consecutive pairs of elements in a list.

E.g.
````
(interpose '+ '(1 2 3)) ~~> '(1 + 2 + 3)
(interpose '$ '(4 (3 2) 1)) ~~> '(4 $ (3 2) $ 1)
````

Of course you also need to realize that the `algebraic->infix` function is recursive, in the sense that every subexpression of an algebraic expression should also be converted to the infix form. This is obvious since you should have noticed that the algebraic expressions are structurally trees.

In [34]:
(define (algebraic->infix e)
  (cond [(number? e) e]
        [(variable? e) e]
        [(sum? e) (interpose '+ (map algebraic->infix (terms e)))]
        [(product? e) (interpose '* (map algebraic->infix (factors e)))]
        [(exponentiation? e) (list (base e) '** (exponent e))]))

(define (deriv-pp e var) (algebraic->infix (deriv e var)))

In [35]:
(deriv-pp '(+ (* 3 x) (* 4 x) (* (** x 2) y)) 'x)

(3 + 4 + ((2 * x) * y))

In [36]:
(deriv-pp '(* (+ (* 3 x) (* 4 x) (* (** x 2) y)) y x) 'x)

(((3 + 4 + ((2 * x) * y)) * y * x) + (((3 * x) + (4 * x) + ((x ** 2) * y)) * y))

### Exercise 2.58 (b)

Actually this is known as the "parsing problem". It is known to be possible to parse infix algebraic expressions and the language of those expressions are known to be in the Context Free Language class.

An implementation for the parser is skipped here for now. If you're interested, consult the Internet.

## §§2.3.3 Example: Representing Sets

In [37]:
%include "/library/prelude.scm"

In [38]:
;; let's plan what operations we want for the set ADT
(if #t #f
 (list
  (empty-set) ; somehow there's no mention of empty-set in the book, but
              ; we should need it
  (make-set xs) ; this will come convenient for trying other operations
  (ajoin-set x set)
  (union-set set1 set2)
  (intersection-set set1 set2)
  (element-of-set? x set)
  (empty-set? set))) ; empty-set? is also something we should have

#f

### Sets as unordered lists (also Exercise 2.59)

In [39]:
(define (empty-set) '())
(define (ajoin-set x set)
  (if (element-of-set? x set) set
      (cons x set)))
(define (union-set set1 set2)
  (cond [(null? set1) set2]
        [(null? set2) set1]
        [else (union-set (cdr set1)
                         (ajoin-set (car set1) set2))]))
(define (intersection-set set1 set2)
  (cond [(or (null? set1) (null? set2)) (empty-set)]
        [else (filter (curry2 element-of-set? set1) set2)]))
(define (element-of-set? x set)
  (exists (curry equal? x) set))
(define (empty-set? set) (null? set))

### Exercise 2.60

In [40]:
(define (make-set xs)
  (define (F xs acc)
    (if (null? xs) acc
        (F (cdr xs) (ajoin-set (car xs) acc))))
  (F xs (empty-set)))

Yes. If we want to prioritize the `ajoin-set` operation, then a duplicate-allowed list representation would be more efficient ($O(1)$ vs. $O(n)$).

#### Tryouts

In [41]:
(make-set (range 10))

(9 8 7 6 5 4 3 2 1 0)

In [42]:
(define X (make-set '(1 2 3 3 2 1 2 4 3 2)))
(define Y (make-set '(4 5 6 4 5 3)))
(list X Y)

((4 3 2 1) (3 6 5 4))

In [43]:
(union-set X Y)

(1 2 3 6 5 4)

In [44]:
(intersection-set X Y)

(3 4)

In [45]:
(ajoin-set 'a X)

(a 4 3 2 1)

In [46]:
(ajoin-set 2 X)

(4 3 2 1)

### Sets as orderd lists (also Exercise 2.61 & 2.62)

We use `eqv?` instead of `=` as in the book so that our version could deal with both numbers and symbols (hopefully also some other primitive data types).

In [47]:
(define (make-set xs) (uniq (sort < xs)))
(define (element-of-set? x set)
  (cond [(null? set) #f]
        [(eqv? x (car set)) #t]
        [(< x (car set)) #f]
        [else (element-of-set? x (cdr set))]))
(define (ajoin-set x set)
  (cond [(null? set) (list x)]
        [(eqv? x (car set)) set]
        [(< x (car set)) (cons x set)]
        [(> x (car set)) (cons (car set)
                               (ajoin-set x (cdr set)))]))
(define (intersection-set set1 set2)
  (cond [(or (null? set1) (null? set2)) (empty-set)]
        [(eqv? (car set1) (car set2)) (cons (car set1)
                                            (intersection-set
                                              (cdr set1)
                                              (cdr set2)))]
        [(< (car set1) (car set2)) (intersection-set (cdr set1) set2)]
        [(> (car set1) (car set2)) (intersection-set set1 (cdr set2))]))
(define (union-set set1 set2)
  (cond [(null? set1) set2]
        [(null? set2) set1]
        [(eqv? (car set1) (car set2)) (cons (car set1)
                                            (union-set (cdr set1)
                                                       (cdr set2)))]
        [(< (car set1) (car set2)) (cons (car set1)
                                         (union-set (cdr set1) set2))]
        [(> (car set1) (car set2)) (cons (car set2)
                                         (union-set set1 (cdr set2)))]))

(define (uniq-gen equiv sorted-list)
  (define (F xs acc)
    (cond [(null? xs) acc]
          [(null? acc) (F (cdr xs) (list (car xs)))]
          [else (F (cdr xs)
                   (if (equiv (car xs) (car acc)) acc
                       (cons (car xs) acc)))]))
  (reverse (F sorted-list '())))
(define uniq (curry uniq-gen equal?))

In [48]:
(make-set (range 10))

(0 1 2 3 4 5 6 7 8 9)

In [49]:
(define X (make-set '(1 2 3 3 2 1 2 4 3 2)))
(define Y (make-set '(4 5 6 4 5 3)))
(list X Y)

((1 2 3 4) (3 4 5 6))

In [50]:
(union-set X Y)

(1 2 3 4 5 6)

In [51]:
(intersection-set X Y)

(3 4)

In [52]:
(ajoin-set 100 X)

(1 2 3 4 100)

In [53]:
(ajoin-set 2 X)

(1 2 3 4)

### Sets as binary trees

In [54]:
;; instead of left/right-branch, we call them left/right-child
;; and use lch/rch to refer to them

(define (make-tree entry lch rch) (list entry lch rch))
(define (entry tree) (car tree))
(define (lch tree) (cadr tree))
(define (rch tree) (caddr tree))
(define (leaf? tree) (and (pair? tree)
                          (null? (lch tree))
                          (null? (rch tree))))

(define (element-of-set? x set)
  (cond [(null? set) #f]
        [(eqv? x (entry set)) #t]
        [(< x (entry set)) (element-of-set? x (lch set))]
        [(> x (entry set)) (element-of-set? x (rch set))]))
(define (ajoin-set x set)
  (cond [(null? set) (make-tree x '() '())]
        [(eqv? x (entry set)) set]
        [(< x (entry set))
         (make-tree (entry set)
                    (ajoin-set x (lch set)) (rch set))]
        [(> x (entry set))
         (make-tree (entry set)
                    (lch set) (ajoin-set x (rch set)))]))

#### Exercise 2.63 & 2.64

So I'm actually giving my implementation of `tree->list` and `list->tree` here. (And it happens that my `tree->list` implementation is the same as `tree->list-2`)

To answer ex2.63 (a): they are the same, and they will output
`(1 3 5 7 9 11)` for the trees in Figure 2.16. (b): apparently the second version is faster, due to the fact that firstly it never uses `append`, which is a very expensive operation, and secondly it is a tail-recursive call.

To answer ex2.64: I don't bother to read the program on the book (it's so ugly!) since I have a better one :D

In [55]:
(define (make-set xs) (list->tree (uniq (sort < xs))))
(define (tree->list tree)
  (define (F tree acc)
    (if (null? tree) acc
        (F (lch tree)
           (cons (entry tree) (F (rch tree) acc)))))
  (F tree '()))
(define (list->tree xs)
  (define (F xs len)
    (cond [(null? xs) '()]
          [else
           (apply (fn (as x bs) (make-tree x as bs))
            (fold-split-map (fn (ys) (F ys (length ys)))
                            xs len))]))
  (define (fold-split xs len)
    (apply (fn (as bs) (list as (car bs) (cdr bs)))
           (split (quotient len 2) xs)))
  (define (fold-split-map f xs len)
    (apply (fn (as x bs) (list (f as) x (f bs)))
           (fold-split xs len)))
  (F xs (length xs)))

(define (split n xs)
  (define (F n xs ys)
    (if (= n 0) (list (reverse ys) xs)
        (F (- n 1) (cdr xs) (cons (car xs) ys))))
  (F n xs '()))
(define (tree-singleton x) (make-tree x '() '()))

#### Exercise 2.65

In [56]:
(define (bst-intersection-set set1 set2)
  (cond [(or (null? set1) (null? set2)) (empty-set)]
        [(eqv? (car set1) (car set2)) (cons (car set1)
                                            (bst-intersection-set
                                              (cdr set1)
                                              (cdr set2)))]
        [(< (car set1) (car set2))
         (bst-intersection-set (cdr set1) set2)]
        [(> (car set1) (car set2))
         (bst-intersection-set set1 (cdr set2))]))
(define (bst-union-set set1 set2)
  (cond [(null? set1) set2]
        [(null? set2) set1]
        [(eqv? (car set1) (car set2))
         (cons (car set1)
               (bst-union-set (cdr set1)
                              (cdr set2)))]
        [(< (car set1) (car set2))
         (cons (car set1) (bst-union-set (cdr set1) set2))]
        [(> (car set1) (car set2))
         (cons (car set2) (bst-union-set set1 (cdr set2)))]))

(define (union-set set1 set2)
    (list->tree (bst-union-set (tree->list set1)
                               (tree->list set2))))
(define (intersection-set set1 set2)
    (list->tree (bst-intersection-set (tree->list set1)
                                      (tree->list set2))))

In [57]:
(make-set (range 10))

(5 (2 (1 (0 () ()) ()) (4 (3 () ()) ())) (8 (7 (6 () ()) ()) (9 () ())))

In [58]:
(define X (make-set '(1 2 3 3 2 1 2 4 3 2)))
(define Y (make-set '(4 5 6 4 5 3)))
(list X Y)

((3 (2 (1 () ()) ()) (4 () ())) (5 (4 (3 () ()) ()) (6 () ())))

In [59]:
(list (tree->list X) (tree->list Y))

((1 2 3 4) (3 4 5 6))

In [60]:
(tree->list (union-set X Y))

(1 2 3 4 5 6)

In [61]:
(tree->list (intersection-set X Y))

(3 4)

In [62]:
(tree->list (ajoin-set 100 X))

(1 2 3 4 100)

In [63]:
(tree->list (ajoin-set 2 X))

(1 2 3 4)

### Sets and information retrieval (also Exercise 2.66)

In [64]:
;; let's first list the procedures that we want
(if #t #t
  (list
   (make-record key data)
   (key record)
   (data record)
   (same-key? record1 record2)
   
   (empty-database)
   (add-record record database) ; we want this operation to also override
                                ; the existing record with the same key
   (make-database list-of-records) ; no guarentee which records would make
                                   ; into the database if records with same
                                   ; keys exist
   
   (record-exists? key database)
   (retrieve-record key database)
   (retrieve-data key database))) ; aka lookup

#t

In [65]:
;; we use a binary search tree to represent a database
;; and a pair to represent a record

;; in contrast to what the book requires, we also allow the keys of records
;; to be symbols. but it is not allowed to mix symbolic keys and numeral
;; keys in a same database
(define (make-record ky data) (cons ky data))
(define (key record) (car record))
(define (data record) (cdr record))
(define (key-relation? rel r1 r2) (rel (key r1) (key r2)))
(define (same-key? r1 r2) (key-relation? eqv? r1 r2))

(define (empty-database) '())
(define (add-record record database)
  (cond [(null? database) (tree-singleton record)]
        [(same-key? record (entry database))
         (make-tree record (lch database) (rch database))]
        [(key-relation? <g record (entry database))
         (make-tree (entry database)
                    (add-record record (lch database)) (rch database))]
        [(key-relation? >g record (entry database))
         (make-tree (entry database)
                    (lch database) (add-record record (rch database)))]))
(define (make-database rs)
  (list->tree (uniq-gen same-key?
                        (sort (curry key-relation? <g) rs))))

(define (record-exists? ky database)
  (if (retrieve-record ky database) #t #f))
(define (retrieve-record ky database)
  (cond [(null? database) #f]
        [(eqv? ky (key (entry database))) (entry database)]
        [(<g ky (key (entry database))) (retrieve-record ky (lch database))]
        [(>g ky (key (entry database))) (retrieve-record ky (rch database))]))
(define (retrieve-data ky database)
  (let ([record (retrieve-record ky database)])
    (if record (data record) #f)))
(define lookup retrieve-data) ; an alias

(define (make-gen-comp comp)
  (fn xs (cond [(forall number? xs) (apply comp xs)]
               [(forall symbol? xs) (apply comp (map symbol->string xs))]
               [else (error "not supported comparison -- GEN-COMP" xs)])))
(define <g (make-gen-comp <))
(define >g (make-gen-comp >))

In [66]:
(define colors (make-database
            '((black . "#000000")
              (white . "#FFFFFF")
              (grey . "#999999")
              (red . "#FF0000")
              (blue . "#0000FF")
              (green . "#00FF00")
              (indianred . "#CD5C5C")
              (olive . "#808000"))))
colors

((indianred . "#CD5C5C") ((green . "#00FF00") ((blue . "#0000FF") ((black . "#000000") () ()) ()) ((grey . "#999999") () ())) ((red . "#FF0000") ((olive . "#808000") () ()) ((white . "#FFFFFF") () ())))

In [67]:
(list (lookup 'green colors)
      (lookup 'olive colors)
      (lookup 'meow colors))

("#00FF00" "#808000" #f)

In [68]:
(lookup 'meow (add-record (make-record 'meow "<RAINBOW!>") colors))

"<RAINBOW!>"

## §§2.3.4 Example: Huffman Encoding Trees

In [69]:
%include "/library/prelude.scm"

In [70]:
;; as usual, we firstly list the procedures we need
(if #t (void)
    (list
     (make-leaf symbol weight)
     (leaf? tr)
     (leaf-symbol tr)
     (leaf-weight tr)

     (make-node tr1 tr2)
     (node-symbols tr)
     (node-weight tr)
     (left-child tr)
     (right-child tr)

     (symbols tr)
     (weight tr)))

(if #t (void)
    (list
     (decode tree bits)       ; since tree is more likely to be fixed,
                              ; we change the order of arguments
                              ; wrt. the book
     (encode tree string)))

#### The tree ADT

In [71]:
;; we represent a leaf as a list of form ('leaf symbol weight)
(define (make-leaf symbol weight)
  (list 'leaf symbol weight))
(define (leaf? tr) (and (pair? tr) (eq? 'leaf (car tr))))
(define (leaf-symbol tr) (cadr tr))
(define (leaf-weight tr) (caddr tr))

;; we represent a node as a list of form (symbols weight lch rch)
;; where lch/rch are left/right-child
(define (make-node tr1 tr2)
  (list (append (symbols tr1) (symbols tr2))
        (+ (weight tr1) (weight tr2))
        tr1 tr2))
(define (node-symbols tr) (car tr))
(define (node-weight tr) (cadr tr))
(define (left-child tr) (caddr tr))
(define (right-child tr) (cadddr tr))

;; the (.< f g) is a macro that expands to (lambda (x) (f (g x))),
;; aka function composition
(define (symbols tr)
  ((if (leaf? tr) (.< singleton leaf-symbol) node-symbols)
   tr))
(define (weight tr)
  (if (leaf? tr) (leaf-weight tr)
      (sum (map (fn (f) (weight (f tr)))
                (list left-child right-child)))))
(define (sum xs) (foldl + 0 xs))

(define (singleton x) (list x))

#### The encode/decode procedures and make-huffman procedure (solutions to Exercise 2.68 & 2.69)

In [72]:
(define (decode tr bits)
  (define (decode-one tr bits) ; returns (symbol rest-bits)
    (if (leaf? tr) (list (leaf-symbol tr) bits)
        (cond [(= 0 (car bits))
               (decode-one (left-child tr) (cdr bits))]
              [(= 1 (car bits))
               (decode-one (right-child tr) (cdr bits))])))
  (define (F bits acc)
    (if (null? bits) acc
        (apply (fn (sym rest) (F rest (cons sym acc)))
               (decode-one tr bits))))
  (reverse (F bits '())))

(define (encode tr msg)
  (define (dfs tr bits acc) ; returns ((sym . rev-encoding) ...)
    (if (leaf? tr)
        (cons (cons (leaf-symbol tr) bits) acc)
        (dfs (left-child tr) (cons 0 bits)
             (dfs (right-child tr) (cons 1 bits) acc))))
  (define table (dfs tr '() '()))
  (define (lookup-sym table sym)
    (if (eq? sym (caar table)) (cdar table)
        (lookup-sym (cdr table) sym)))
  (define (F msg acc)
    (if (null? msg) acc
        (F (cdr msg) (append (lookup-sym table (car msg)) acc))))
  (reverse (F msg '())))

In [73]:
(define (adjoin-queue tr queue)
  (cond [(null? queue) (singleton tr)]
        [(< (weight tr) (weight (car queue)))
         (cons tr queue)]
        [else (cons (car queue) (adjoin-queue tr (cdr queue)))]))

(define (make-huffman syms) ; syms in the form ((sym . freq) ...)
  (define init-queue
    (map (fn (p) (make-leaf (car p) (cdr p)))
         (sort (fn (p1 p2) (< (cdr p1) (cdr p2))) syms)))
  (define (merge queue)
    (let ([tr1 (car queue)]
          [tr2 (cadr queue)])
      (adjoin-queue (make-node tr1 tr2)
                    (cddr queue))))
  (define (F queue)
    (if (singleton? queue) queue
        (F (merge queue))))
  (define (singleton? queue) (and (pair? queue) (null? (cdr queue))))
  (car (F init-queue)))

#### Tryouts

In [74]:
(define huffman1
  (make-huffman '((A . 4) (B . 2) (C . 1) (D . 1))))
huffman1

((A B C D) 8 (leaf A 4) ((B C D) 4 (leaf B 2) ((C D) 2 (leaf C 1) (leaf D 1))))

In [75]:
(define encode1 (curry encode huffman1))
(define decode1 (curry decode huffman1))

In [76]:
(encode1 '(A A B A B C D B A B C))

(0 0 1 0 0 1 0 1 1 0 1 1 1 1 0 0 1 0 1 1 0)

In [77]:
((.< decode1 encode1) '(A A B A B C D B A B C))

(A A B A B C D B A B C)

### Exercise 2.67

In [78]:
(define huffman67
  (make-node
   (make-leaf 'A 4)
   (make-node
    (make-leaf 'B 2)
    (make-node
     (make-leaf 'D 1)
     (make-leaf 'C 1)))))
(define message67 '(0 1 1 0 0 1 0 1 0 1 1 1 0))

In [79]:
(decode huffman67 message67)

(A D A B B C A)

In [80]:
(encode huffman67 '(A D A B B C A))

(0 1 1 0 0 1 0 1 0 1 1 1 0)

### Exercise 2.70

In [81]:
(define huffman70
  (make-huffman
   '((a . 2) (na . 16) (boom . 1) (sha . 3) (get . 2)
     (yip . 9) (job . 2) (wah . 1))))
(define message70
  '(get a job
    sha na na na na na na na na
    get a job
    sha na na na na na na na na
    wah yip yip yip yip yip yip yip yip yip
    sha boom))

In [82]:
(length (encode huffman70 message70))

84

In [83]:
;; we need 3 bits for each symbol
(* 3 (length message70))

108

### Exercise 2.71

Sketches are skipped.

It would require 1 bit to encode the most frequent symbol
and the least frequent symbol would require n-1 bits to encode

### Exercise 2.72

It takes $O(1)$ for the most frequent symbol and $O(n)$ for the least
frequent symbol. This could be improved a lot if we replace the
`lookup-sym` mechanism with a BST, which will reduce the encoding of
each symbol to $O(\log(n))$